In [1]:
import torch as t
from torch import nn
import numpy as np

In [2]:
class Conv2dBlock(nn.Module):
    def __init__(self,ninp,fmaps,kwidth,stride):
        super().__init__()
        self.conv=nn.Conv2d(ninp,fmaps,kwidth,stride)
        self.norm = build_norm_layer('bnorm', self.conv, fmaps)
        self.pool=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
        self.act=nn.ELU()
        
    def forward(self,x):
        x=self.conv(x)
        x=self.pool(x)
        x=self.norm(x)
        x=self.act(x)
        return x

    

In [3]:

import pdb


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2dBlock') != -1:
        print('Initializing weights of convresblock to 0.0, 0.02')
        for k, p in m.named_parameters():
            if 'weight' in k and 'conv' in k:
                p.data.normal_(0.0, 0.02)
    elif classname.find('Conv2d') != -1:
        print('Initialzing weight to 0.0, 0.02 for module: ', m)
        m.weight.data.normal_(0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            print('bias to 0 for module: ', m)
            m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        print('Initializing FC weight to xavier uniform')
        nn.init.xavier_uniform_(m.weight.data)
def build_norm_layer(norm_type, param=None, num_feats=None):
    if norm_type == 'bnorm':
        return nn.BatchNorm2d(num_feats)
    elif norm_type == 'snorm':
        spectral_norm(param)
        return None
    elif norm_type is None:
        return None
    else:
        raise TypeError('Unrecognized norm type: ', norm_type)

class PPGNet(nn.Module):
    """
    网络结构定义
    """
    
    def __init__(self,ntr,nc):
        super(PPGNet,self).__init__()
#       ngf=opt.ngf # 
        self.norm_type='bnorm'
        self.ntr=ntr #每一段信号追踪的分片数
        self.nc=nc #中间卷积层的个数
        self.ConvBlocks=nn.ModuleList()
        self.conv1=nn.Conv2d(4,8,1)
        self.conv2=nn.Conv2d(8,16,(ntr,3))
        self.norm2 = build_norm_layer(self.norm_type, self.conv2, 16)
        self.act2=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
        for i in np.arange(nc):
            ConvBlock=Conv2dBlock(2**(i+4),2**(i+5),(1,3),(1,1))
            self.ConvBlocks.append(ConvBlock)
        self.conv3=nn.Conv2d(2**(nc+4),32,(1,1),(1,1))
        self.norm3 = build_norm_layer(self.norm_type, self.conv2, 32)
        self.act3=nn.ELU()
        self.fc=nn.Sequential(
            nn.Linear(64,512),
            nn.BatchNorm1d(1),
            nn.ELU(),
            nn.Dropout(),
            nn.Linear(512,1),
            nn.ELU()
        )

    def forward(self,x):
#         print('1:',x.shape)
        x=self.conv1(x)
#         print('2:',x.shape)
        x=self.conv2(x)
#         print('3:',x.shape)
        x=self.norm2(x)
        x=self.act2(x)
#         print('4:',x.shape)
#         pdb.set_trace()
        for i in np.arange(self.nc):
            ConvBlock=self.ConvBlocks[i]
            x=ConvBlock(x)
#             print('{}:'.format(i+5),x.shape)
#         pdb.set_trace()
        x=self.conv3(x)
        x=self.norm3(x)
        x=self.act3(x)
        x=x.view(x.size(0),1,-1)
        out=self.fc(x)
        return out.squeeze()
    
        

import argparse
import os
import json
parser = argparse.ArgumentParser()
parser.add_argument('--ngf', type=int, default=None,help="original input channel")
parser.add_argument('--nc', type=int, default=7,help="number of mid feature maps")
parser.add_argument('--save_path', type=str, default='saveconfig',help="path to save config")
opts = parser.parse_args()
if not os.path.exists(opts.save_path):
    os.makedirs(opts.save_path)
# save opts
with open(os.path.join(opts.save_path, 'train.opts'), 'w') as cfg_f:
    cfg_f.write(json.dumps(vars(opts), indent=2))
print('Parsed arguments: ', json.dumps(vars(opts), indent=2))

In [4]:
Net=PPGNet(7,7)
Net.apply(weights_init)
input=t.randn(2,4,7,1025)
# print(Net)

Initialzing weight to 0.0, 0.02 for module:  Conv2d(16, 32, kernel_size=(1, 3), stride=(1, 1))
bias to 0 for module:  Conv2d(16, 32, kernel_size=(1, 3), stride=(1, 1))
Initializing weights of convresblock to 0.0, 0.02
Initialzing weight to 0.0, 0.02 for module:  Conv2d(32, 64, kernel_size=(1, 3), stride=(1, 1))
bias to 0 for module:  Conv2d(32, 64, kernel_size=(1, 3), stride=(1, 1))
Initializing weights of convresblock to 0.0, 0.02
Initialzing weight to 0.0, 0.02 for module:  Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1))
bias to 0 for module:  Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1))
Initializing weights of convresblock to 0.0, 0.02
Initialzing weight to 0.0, 0.02 for module:  Conv2d(128, 256, kernel_size=(1, 3), stride=(1, 1))
bias to 0 for module:  Conv2d(128, 256, kernel_size=(1, 3), stride=(1, 1))
Initializing weights of convresblock to 0.0, 0.02
Initialzing weight to 0.0, 0.02 for module:  Conv2d(256, 512, kernel_size=(1, 3), stride=(1, 1))
bias to 0 for module:  Co

In [9]:
print(Net)

PPGNet(
  (ConvBlocks): ModuleList(
    (0): Conv2dBlock(
      (conv): Conv2d(16, 32, kernel_size=(1, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (act): ELU(alpha=1.0)
    )
    (1): Conv2dBlock(
      (conv): Conv2d(32, 64, kernel_size=(1, 3), stride=(1, 1))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (act): ELU(alpha=1.0)
    )
    (2): Conv2dBlock(
      (conv): Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1))
      (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (act): ELU(alpha=1.0)
    )
    (3): Conv2dBlock(
 

In [20]:
output=Net(input)
print(output)

tensor([0.4817, 3.1979], grad_fn=<SqueezeBackward0>)


In [21]:
output.shape
output= output.squeeze()
output

tensor([0.4817, 3.1979], grad_fn=<SqueezeBackward0>)

In [22]:
target=t.Tensor([82,85])
criterion=nn.L1Loss()
loss=criterion(output,target)
loss

tensor(81.6602, grad_fn=<L1LossBackward>)

In [23]:
Net.zero_grad()
loss.backward()

In [24]:
import torch.optim as optim
optimizer=optim.Adam(Net.parameters(),lr=0.00001)



In [25]:
from torch.utils import data
import os
import scipy.io as sio
class PPGData(data.Dataset):
    def __init__(self,root):
        datas=os.listdir(root)
        self.totaldata=[os.path.join(root,data) for data in datas]
    def __getitem__(self,index):
        freqdata=np.zeros((4,7,1025))
        ppgpath=self.totaldata[index]
        freq=sio.loadmat(ppgpath)
        freq=freq['datasample']
        for i in np.arange(freq.size):
            freqdata[i]=freq[i][0]
        listslice=ppgpath.split('-')
        label=listslice[2][:-4]
        return freqdata,label
    def __len__(self):
        return len(self.totaldata)
        

In [26]:
import matplotlib.pyplot as plt
fs=np.arange(1025)*125/1025
plt.rcParams['figure.figsize'] = [20, 30]
filepath='/home/wcj/PytorchDemo/Data'
dataloader=PPGData(filepath)
for data,label in dataloader:
#     pdb.set_trace()
#     print(data.shape)
#     print(label)
#     plt.subplot(711)
#     plt.plot(fs[1:128],data[0][0][1:128])
#     plt.subplot(712)
#     plt.plot(fs[1:128],data[0][1][1:128])
#     plt.subplot(713)
#     plt.plot(fs[1:128],data[0][2][1:128])
#     plt.subplot(714)
#     plt.plot(fs[1:128],data[0][3][1:128])
#     plt.subplot(715)
#     plt.plot(fs[1:128],data[0][4][1:128])
#     plt.subplot(716)
#     plt.plot(fs[1:128],data[0][5][1:128])
#     plt.subplot(717)
#     plt.plot(fs[1:128],data[0][6][1:128])
    break
    

In [27]:
for data,label in dataloader:
#     pdb.set_trace()
#     print(data.shape)
#     print(label)
#     plt.subplot(411)
#     plt.plot(fs[1:128],data[0][0][1:128])
#     plt.subplot(412)
#     plt.plot(fs[1:128],data[1][0][1:128])
#     plt.subplot(413)
#     plt.plot(fs[1:128],data[2][0][1:128])
#     plt.subplot(414)
#     plt.plot(fs[1:128],data[3][0][1:128])
    break

In [28]:

from torch.utils.data import DataLoader
# pdb.set_trace()
device = 'cpu'
if t.cuda.is_available:
#     device = t.device('cuda:0' if t.cuda.is_available() else 'cpu')
    device = 'cuda'
CUDA = (device == 'cuda')
print(device,CUDA)
# pdb.set_trace()
dloador=DataLoader(dataloader,batch_size=10,shuffle=True,num_workers=2,pin_memory=CUDA)

print(len(dataloader))
dataiter=iter(dloador)
sample,label=dataiter.next()
# print(label)

cuda True
1556


In [29]:
##################
# Leave one  lr=1e-5
##################
Net=Net.double()
for epoch in np.arange(200):
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         pdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/PytorchDemo/checkpoints/0327Net_epoch%s_LeaveOne.pth'%(epoch+1))

[1,   10] loss 136.927907
[1,   20] loss 134.336946
[1,   30] loss 130.630981
[1,   40] loss 127.388860
[1,   50] loss 131.424074
[1,   60] loss 130.455238
[1,   70] loss 128.238995
[1,   80] loss 131.476412
[1,   90] loss 126.405525
[1,  100] loss 129.794954
[1,  110] loss 126.908878
[1,  120] loss 128.332965
[1,  130] loss 131.408541
[1,  140] loss 127.228939
[1,  150] loss 123.502011
[2,   10] loss 127.100142
[2,   20] loss 127.222714
[2,   30] loss 125.183928
[2,   40] loss 127.380532
[2,   50] loss 125.835229
[2,   60] loss 125.200181
[2,   70] loss 124.875661
[2,   80] loss 124.688729
[2,   90] loss 119.781240
[2,  100] loss 126.732308
[2,  110] loss 120.968304
[2,  120] loss 118.297589
[2,  130] loss 123.372982
[2,  140] loss 121.741574
[2,  150] loss 120.963223
[3,   10] loss 123.951611
[3,   20] loss 119.104416
[3,   30] loss 123.269537
[3,   40] loss 123.448904
[3,   50] loss 118.945016
[3,   60] loss 117.067598
[3,   70] loss 116.867208
[3,   80] loss 120.492510
[3,   90] lo

[21,  140] loss 96.296765
[21,  150] loss 97.583469
[22,   10] loss 91.600762
[22,   20] loss 92.712752
[22,   30] loss 93.532563
[22,   40] loss 94.872251
[22,   50] loss 93.306295
[22,   60] loss 94.968467
[22,   70] loss 95.841716
[22,   80] loss 92.251405
[22,   90] loss 93.425723
[22,  100] loss 98.602198
[22,  110] loss 93.655217
[22,  120] loss 96.106634
[22,  130] loss 95.294716
[22,  140] loss 95.815153
[22,  150] loss 95.624384
[23,   10] loss 93.600698
[23,   20] loss 94.138753
[23,   30] loss 89.926943
[23,   40] loss 96.570005
[23,   50] loss 93.178113
[23,   60] loss 90.296065
[23,   70] loss 92.883209
[23,   80] loss 88.829248
[23,   90] loss 97.795416
[23,  100] loss 97.333259
[23,  110] loss 94.035702
[23,  120] loss 94.431427
[23,  130] loss 93.512857
[23,  140] loss 96.218149
[23,  150] loss 91.969282
[24,   10] loss 91.974195
[24,   20] loss 93.343076
[24,   30] loss 95.930511
[24,   40] loss 92.499888
[24,   50] loss 88.626786
[24,   60] loss 96.524870
[24,   70] l

[42,  150] loss 77.340254
[43,   10] loss 81.420795
[43,   20] loss 79.573786
[43,   30] loss 81.215716
[43,   40] loss 76.343065
[43,   50] loss 80.328920
[43,   60] loss 80.515914
[43,   70] loss 76.773160
[43,   80] loss 80.296237
[43,   90] loss 74.804688
[43,  100] loss 80.126579
[43,  110] loss 80.209956
[43,  120] loss 76.584858
[43,  130] loss 76.234843
[43,  140] loss 72.799383
[43,  150] loss 79.242847
[44,   10] loss 79.235145
[44,   20] loss 78.540344
[44,   30] loss 79.787212
[44,   40] loss 80.481338
[44,   50] loss 77.548051
[44,   60] loss 78.794813
[44,   70] loss 77.480359
[44,   80] loss 75.702855
[44,   90] loss 81.027696
[44,  100] loss 75.518549
[44,  110] loss 74.971796
[44,  120] loss 79.248354
[44,  130] loss 76.014520
[44,  140] loss 78.442532
[44,  150] loss 76.575141
[45,   10] loss 80.945283
[45,   20] loss 73.363971
[45,   30] loss 78.989476
[45,   40] loss 74.496400
[45,   50] loss 79.424092
[45,   60] loss 74.324944
[45,   70] loss 77.961194
[45,   80] l

[64,   10] loss 61.248732
[64,   20] loss 65.597643
[64,   30] loss 61.028328
[64,   40] loss 57.523925
[64,   50] loss 61.596877
[64,   60] loss 64.782086
[64,   70] loss 58.767653
[64,   80] loss 63.832383
[64,   90] loss 63.326603
[64,  100] loss 66.317253
[64,  110] loss 59.002697
[64,  120] loss 57.225034
[64,  130] loss 63.765148
[64,  140] loss 64.368300
[64,  150] loss 60.522733
[65,   10] loss 62.820846
[65,   20] loss 62.625279
[65,   30] loss 61.377850
[65,   40] loss 64.158181
[65,   50] loss 63.115996
[65,   60] loss 59.185690
[65,   70] loss 57.566158
[65,   80] loss 55.951257
[65,   90] loss 59.122128
[65,  100] loss 64.284093
[65,  110] loss 59.261162
[65,  120] loss 65.710667
[65,  130] loss 58.966196
[65,  140] loss 60.986201
[65,  150] loss 57.588430
[66,   10] loss 66.385722
[66,   20] loss 57.392012
[66,   30] loss 61.747683
[66,   40] loss 60.373182
[66,   50] loss 55.444037
[66,   60] loss 59.891560
[66,   70] loss 62.084187
[66,   80] loss 61.799642
[66,   90] l

[85,   20] loss 45.216280
[85,   30] loss 43.748449
[85,   40] loss 45.481549
[85,   50] loss 43.883980
[85,   60] loss 44.644843
[85,   70] loss 44.072246
[85,   80] loss 45.058078
[85,   90] loss 45.129353
[85,  100] loss 41.761980
[85,  110] loss 45.454467
[85,  120] loss 48.919676
[85,  130] loss 43.278654
[85,  140] loss 45.839304
[85,  150] loss 43.399086
[86,   10] loss 41.905585
[86,   20] loss 45.166877
[86,   30] loss 47.878753
[86,   40] loss 49.114441
[86,   50] loss 37.771826
[86,   60] loss 44.971690
[86,   70] loss 43.791619
[86,   80] loss 45.898269
[86,   90] loss 40.297257
[86,  100] loss 39.974247
[86,  110] loss 44.262661
[86,  120] loss 45.592736
[86,  130] loss 42.367413
[86,  140] loss 45.263497
[86,  150] loss 42.670186
[87,   10] loss 40.752162
[87,   20] loss 41.156781
[87,   30] loss 44.294095
[87,   40] loss 46.180099
[87,   50] loss 43.422077
[87,   60] loss 43.360291
[87,   70] loss 39.263638
[87,   80] loss 40.562388
[87,   90] loss 43.820232
[87,  100] l

[105,  140] loss 24.928341
[105,  150] loss 27.306113
[106,   10] loss 24.940195
[106,   20] loss 27.990618
[106,   30] loss 27.395232
[106,   40] loss 26.313770
[106,   50] loss 28.100580
[106,   60] loss 30.716414
[106,   70] loss 26.764624
[106,   80] loss 26.714203
[106,   90] loss 23.083202
[106,  100] loss 24.062083
[106,  110] loss 32.679953
[106,  120] loss 29.049681
[106,  130] loss 28.066471
[106,  140] loss 23.387250
[106,  150] loss 27.867666
[107,   10] loss 29.508254
[107,   20] loss 24.814720
[107,   30] loss 28.872628
[107,   40] loss 27.326467
[107,   50] loss 20.701224
[107,   60] loss 29.016154
[107,   70] loss 28.345180
[107,   80] loss 23.899237
[107,   90] loss 24.793937
[107,  100] loss 26.178712
[107,  110] loss 26.624266
[107,  120] loss 25.329393
[107,  130] loss 25.149690
[107,  140] loss 26.728448
[107,  150] loss 22.786945
[108,   10] loss 28.005265
[108,   20] loss 27.962761
[108,   30] loss 25.273790
[108,   40] loss 24.881469
[108,   50] loss 26.031164
[

[126,   30] loss 12.049702
[126,   40] loss 11.443183
[126,   50] loss 9.751799
[126,   60] loss 11.768137
[126,   70] loss 12.627343
[126,   80] loss 11.589042
[126,   90] loss 11.561786
[126,  100] loss 11.429151
[126,  110] loss 13.899829
[126,  120] loss 16.231353
[126,  130] loss 11.486032
[126,  140] loss 12.924024
[126,  150] loss 11.489875
[127,   10] loss 11.161554
[127,   20] loss 12.246672
[127,   30] loss 12.713261
[127,   40] loss 11.603748
[127,   50] loss 14.897070
[127,   60] loss 10.578070
[127,   70] loss 14.051348
[127,   80] loss 11.914357
[127,   90] loss 11.782454
[127,  100] loss 15.239835
[127,  110] loss 12.246450
[127,  120] loss 11.682511
[127,  130] loss 11.296494
[127,  140] loss 10.169926
[127,  150] loss 12.910134
[128,   10] loss 8.942628
[128,   20] loss 14.655057
[128,   30] loss 15.074436
[128,   40] loss 12.780728
[128,   50] loss 10.383826
[128,   60] loss 10.384622
[128,   70] loss 11.876931
[128,   80] loss 12.529798
[128,   90] loss 9.802986
[128

[146,  130] loss 8.142068
[146,  140] loss 9.160446
[146,  150] loss 8.982042
[147,   10] loss 8.694140
[147,   20] loss 8.498352
[147,   30] loss 7.398466
[147,   40] loss 7.849632
[147,   50] loss 8.780961
[147,   60] loss 7.639033
[147,   70] loss 8.296638
[147,   80] loss 8.639623
[147,   90] loss 6.736647
[147,  100] loss 7.325665
[147,  110] loss 8.071323
[147,  120] loss 7.428212
[147,  130] loss 8.519319
[147,  140] loss 10.108540
[147,  150] loss 9.509853
[148,   10] loss 9.054162
[148,   20] loss 8.032227
[148,   30] loss 7.903290
[148,   40] loss 7.016453
[148,   50] loss 8.627197
[148,   60] loss 8.186963
[148,   70] loss 8.190000
[148,   80] loss 8.540328
[148,   90] loss 8.430356
[148,  100] loss 7.416814
[148,  110] loss 9.643636
[148,  120] loss 9.101690
[148,  130] loss 9.678951
[148,  140] loss 8.049897
[148,  150] loss 8.455890
[149,   10] loss 7.402301
[149,   20] loss 9.839118
[149,   30] loss 8.665534
[149,   40] loss 9.669695
[149,   50] loss 8.780034
[149,   60]

[167,  130] loss 7.402089
[167,  140] loss 9.006366
[167,  150] loss 8.890080
[168,   10] loss 6.725977
[168,   20] loss 8.122415
[168,   30] loss 6.790757
[168,   40] loss 8.077374
[168,   50] loss 8.300708
[168,   60] loss 7.956355
[168,   70] loss 7.905910
[168,   80] loss 7.520829
[168,   90] loss 6.736461
[168,  100] loss 7.960600
[168,  110] loss 7.864659
[168,  120] loss 9.152365
[168,  130] loss 8.184454
[168,  140] loss 7.551494
[168,  150] loss 9.216112
[169,   10] loss 6.786804
[169,   20] loss 7.041751
[169,   30] loss 8.731653
[169,   40] loss 6.475810
[169,   50] loss 8.583293
[169,   60] loss 7.544969
[169,   70] loss 8.545024
[169,   80] loss 6.353537
[169,   90] loss 7.717059
[169,  100] loss 9.035415
[169,  110] loss 7.221088
[169,  120] loss 8.259794
[169,  130] loss 7.709654
[169,  140] loss 7.532559
[169,  150] loss 7.642432
[170,   10] loss 8.206674
[170,   20] loss 6.812887
[170,   30] loss 7.977917
[170,   40] loss 8.201020
[170,   50] loss 7.495172
[170,   60] 

[188,  130] loss 7.691775
[188,  140] loss 8.205500
[188,  150] loss 6.694804
[189,   10] loss 8.059547
[189,   20] loss 6.453197
[189,   30] loss 6.983826
[189,   40] loss 8.553587
[189,   50] loss 7.601033
[189,   60] loss 7.131865
[189,   70] loss 8.580224
[189,   80] loss 7.122560
[189,   90] loss 7.274501
[189,  100] loss 7.028141
[189,  110] loss 8.304797
[189,  120] loss 7.102193
[189,  130] loss 7.712609
[189,  140] loss 7.255114
[189,  150] loss 8.819887
[190,   10] loss 6.537110
[190,   20] loss 6.606705
[190,   30] loss 6.940187
[190,   40] loss 6.777559
[190,   50] loss 7.307320
[190,   60] loss 8.479007
[190,   70] loss 6.913210
[190,   80] loss 7.380270
[190,   90] loss 7.395692
[190,  100] loss 6.778426
[190,  110] loss 8.447677
[190,  120] loss 7.750732
[190,  130] loss 6.577756
[190,  140] loss 6.937367
[190,  150] loss 7.050186
[191,   10] loss 6.598335
[191,   20] loss 7.139917
[191,   30] loss 7.069449
[191,   40] loss 7.684694
[191,   50] loss 7.232607
[191,   60] 

In [14]:
##################
# Leave one 
##################
Net=Net.double()
for epoch in np.arange(200):
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         pdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/PytorchDemo/checkpoints/Net_epoch%s_LeaveOne.pth'%(epoch+1))

[1,   10] loss 132.045896
[1,   20] loss 126.837905
[1,   30] loss 128.985516
[1,   40] loss 126.218905
[1,   50] loss 128.378921
[1,   60] loss 124.884921
[1,   70] loss 122.104294
[1,   80] loss 119.319174
[1,   90] loss 128.219387
[1,  100] loss 119.888792
[1,  110] loss 113.354037
[1,  120] loss 113.339893
[1,  130] loss 114.904323
[1,  140] loss 111.146857
[1,  150] loss 115.321253
[2,   10] loss 110.674001
[2,   20] loss 113.123282
[2,   30] loss 109.906814
[2,   40] loss 105.747814
[2,   50] loss 109.802548
[2,   60] loss 109.237008
[2,   70] loss 108.217723
[2,   80] loss 105.097475
[2,   90] loss 104.383181
[2,  100] loss 106.832186
[2,  110] loss 109.586093
[2,  120] loss 108.338898
[2,  130] loss 103.879660
[2,  140] loss 103.917763
[2,  150] loss 104.844841
[3,   10] loss 108.306006
[3,   20] loss 105.017850
[3,   30] loss 101.669919
[3,   40] loss 99.193763
[3,   50] loss 98.932369
[3,   60] loss 98.441335
[3,   70] loss 102.642142
[3,   80] loss 103.521220
[3,   90] loss 

[22,   50] loss 23.627349
[22,   60] loss 20.863333
[22,   70] loss 20.734784
[22,   80] loss 23.868616
[22,   90] loss 20.184588
[22,  100] loss 21.173510
[22,  110] loss 20.956435
[22,  120] loss 21.971533
[22,  130] loss 20.011063
[22,  140] loss 21.754174
[22,  150] loss 23.796245
[23,   10] loss 19.888557
[23,   20] loss 20.103492
[23,   30] loss 17.910662
[23,   40] loss 20.671639
[23,   50] loss 19.276292
[23,   60] loss 16.344707
[23,   70] loss 21.099073
[23,   80] loss 19.636367
[23,   90] loss 19.303001
[23,  100] loss 20.741773
[23,  110] loss 17.668190
[23,  120] loss 17.188713
[23,  130] loss 15.568742
[23,  140] loss 18.117953
[23,  150] loss 14.407363
[24,   10] loss 20.021112
[24,   20] loss 16.336050
[24,   30] loss 14.557052
[24,   40] loss 15.370735
[24,   50] loss 17.615877
[24,   60] loss 14.391395
[24,   70] loss 14.731660
[24,   80] loss 16.722494
[24,   90] loss 12.647972
[24,  100] loss 17.421299
[24,  110] loss 13.275535
[24,  120] loss 15.423171
[24,  130] l

[43,  150] loss 6.306708
[44,   10] loss 6.453353
[44,   20] loss 7.357032
[44,   30] loss 6.697126
[44,   40] loss 6.490349
[44,   50] loss 7.247010
[44,   60] loss 7.414593
[44,   70] loss 8.241648
[44,   80] loss 7.398372
[44,   90] loss 8.517503
[44,  100] loss 6.339345
[44,  110] loss 8.658704
[44,  120] loss 8.091616
[44,  130] loss 6.846730
[44,  140] loss 6.847270
[44,  150] loss 6.373723
[45,   10] loss 6.192957
[45,   20] loss 8.286673
[45,   30] loss 7.429681
[45,   40] loss 7.386690
[45,   50] loss 7.786263
[45,   60] loss 9.117094
[45,   70] loss 7.112817
[45,   80] loss 7.064933
[45,   90] loss 6.812757
[45,  100] loss 7.500192
[45,  110] loss 7.424465
[45,  120] loss 7.130615
[45,  130] loss 6.205383
[45,  140] loss 7.694835
[45,  150] loss 6.373517
[46,   10] loss 8.041712
[46,   20] loss 6.745634
[46,   30] loss 7.629142
[46,   40] loss 6.775917
[46,   50] loss 6.059740
[46,   60] loss 7.487835
[46,   70] loss 6.836733
[46,   80] loss 7.932667
[46,   90] loss 6.660275


[65,  130] loss 7.055715
[65,  140] loss 6.936762
[65,  150] loss 5.735297
[66,   10] loss 6.495564
[66,   20] loss 6.923835
[66,   30] loss 7.096890
[66,   40] loss 6.854980
[66,   50] loss 5.984787
[66,   60] loss 6.396500
[66,   70] loss 5.915191
[66,   80] loss 7.210884
[66,   90] loss 5.857546
[66,  100] loss 6.893053
[66,  110] loss 6.563463
[66,  120] loss 5.805773
[66,  130] loss 6.686194
[66,  140] loss 6.461449
[66,  150] loss 6.607142
[67,   10] loss 6.640726
[67,   20] loss 7.927695
[67,   30] loss 5.947828
[67,   40] loss 5.876874
[67,   50] loss 6.052107
[67,   60] loss 6.189022
[67,   70] loss 6.051739
[67,   80] loss 5.818416
[67,   90] loss 6.033120
[67,  100] loss 7.102122
[67,  110] loss 6.383132
[67,  120] loss 6.549206
[67,  130] loss 6.551681
[67,  140] loss 6.769994
[67,  150] loss 6.329624
[68,   10] loss 6.304936
[68,   20] loss 7.075672
[68,   30] loss 5.942593
[68,   40] loss 6.494113
[68,   50] loss 6.562782
[68,   60] loss 5.614480
[68,   70] loss 6.739466


[87,  110] loss 6.589915
[87,  120] loss 7.361134
[87,  130] loss 5.931842
[87,  140] loss 5.829300
[87,  150] loss 7.285093
[88,   10] loss 6.109324
[88,   20] loss 5.685261
[88,   30] loss 6.618409
[88,   40] loss 5.708832
[88,   50] loss 7.203723
[88,   60] loss 6.483129
[88,   70] loss 6.592838
[88,   80] loss 6.584230
[88,   90] loss 5.971752
[88,  100] loss 6.817577
[88,  110] loss 6.839373
[88,  120] loss 6.027341
[88,  130] loss 6.313039
[88,  140] loss 6.495398
[88,  150] loss 6.613354
[89,   10] loss 6.136804
[89,   20] loss 6.158083
[89,   30] loss 6.173929
[89,   40] loss 6.421700
[89,   50] loss 6.169310
[89,   60] loss 6.697027
[89,   70] loss 6.141287
[89,   80] loss 7.147425
[89,   90] loss 6.434745
[89,  100] loss 6.055692
[89,  110] loss 6.055386
[89,  120] loss 6.668932
[89,  130] loss 6.599780
[89,  140] loss 6.205818
[89,  150] loss 6.299416
[90,   10] loss 6.300029
[90,   20] loss 6.483112
[90,   30] loss 6.468930
[90,   40] loss 7.077614
[90,   50] loss 6.192254


[109,   40] loss 6.330156
[109,   50] loss 6.334067
[109,   60] loss 6.453744
[109,   70] loss 5.226197
[109,   80] loss 6.482836
[109,   90] loss 6.572899
[109,  100] loss 6.609764
[109,  110] loss 5.905430
[109,  120] loss 5.848636
[109,  130] loss 5.918582
[109,  140] loss 6.156722
[109,  150] loss 5.786685
[110,   10] loss 6.053221
[110,   20] loss 5.848777
[110,   30] loss 6.279015
[110,   40] loss 6.709496
[110,   50] loss 6.540095
[110,   60] loss 6.389815
[110,   70] loss 6.306451
[110,   80] loss 5.521160
[110,   90] loss 6.249465
[110,  100] loss 6.078795
[110,  110] loss 5.716359
[110,  120] loss 6.139959
[110,  130] loss 6.221398
[110,  140] loss 7.143447
[110,  150] loss 5.661645
[111,   10] loss 5.449676
[111,   20] loss 6.228656
[111,   30] loss 6.795597
[111,   40] loss 5.648567
[111,   50] loss 5.871857
[111,   60] loss 6.258431
[111,   70] loss 6.354863
[111,   80] loss 6.751051
[111,   90] loss 7.625388
[111,  100] loss 6.966190
[111,  110] loss 5.850850
[111,  120] 

[130,   50] loss 6.403488
[130,   60] loss 6.476185
[130,   70] loss 6.993796
[130,   80] loss 6.431570
[130,   90] loss 5.898765
[130,  100] loss 5.988431
[130,  110] loss 5.576275
[130,  120] loss 7.045662
[130,  130] loss 6.173989
[130,  140] loss 5.802795
[130,  150] loss 5.893751
[131,   10] loss 6.292550
[131,   20] loss 6.023713
[131,   30] loss 5.743611
[131,   40] loss 6.653432
[131,   50] loss 5.817775
[131,   60] loss 5.721494
[131,   70] loss 5.514494
[131,   80] loss 5.564321
[131,   90] loss 5.921657
[131,  100] loss 6.309942
[131,  110] loss 6.901318
[131,  120] loss 5.732553
[131,  130] loss 5.634410
[131,  140] loss 5.980336
[131,  150] loss 6.040741
[132,   10] loss 5.778607
[132,   20] loss 6.577619
[132,   30] loss 6.179546
[132,   40] loss 5.890872
[132,   50] loss 5.355668
[132,   60] loss 5.820174
[132,   70] loss 6.022896
[132,   80] loss 7.357660
[132,   90] loss 5.907760
[132,  100] loss 6.292014
[132,  110] loss 6.164051
[132,  120] loss 5.362603
[132,  130] 

[151,   60] loss 5.843329
[151,   70] loss 6.557040
[151,   80] loss 5.831230
[151,   90] loss 5.802840
[151,  100] loss 5.989590
[151,  110] loss 4.746139
[151,  120] loss 6.087754
[151,  130] loss 6.773749
[151,  140] loss 7.095319
[151,  150] loss 5.861826
[152,   10] loss 6.015961
[152,   20] loss 4.993670
[152,   30] loss 5.164854
[152,   40] loss 6.762714
[152,   50] loss 5.876300
[152,   60] loss 6.270209
[152,   70] loss 6.013964
[152,   80] loss 5.738779
[152,   90] loss 6.383181
[152,  100] loss 5.805734
[152,  110] loss 5.136914
[152,  120] loss 6.695209
[152,  130] loss 5.810672
[152,  140] loss 6.177126
[152,  150] loss 6.227264
[153,   10] loss 5.704726
[153,   20] loss 6.208890
[153,   30] loss 6.119679
[153,   40] loss 5.224711
[153,   50] loss 6.110191
[153,   60] loss 5.442131
[153,   70] loss 5.553450
[153,   80] loss 5.800659
[153,   90] loss 6.134943
[153,  100] loss 5.870098
[153,  110] loss 5.158084
[153,  120] loss 5.914574
[153,  130] loss 6.555912
[153,  140] 

[172,   70] loss 5.294708
[172,   80] loss 5.307590
[172,   90] loss 6.497504
[172,  100] loss 6.226131
[172,  110] loss 5.887732
[172,  120] loss 5.615278
[172,  130] loss 5.971258
[172,  140] loss 6.054953
[172,  150] loss 5.904047
[173,   10] loss 6.179147
[173,   20] loss 5.416177
[173,   30] loss 5.826511
[173,   40] loss 5.917790
[173,   50] loss 5.173039
[173,   60] loss 5.938402
[173,   70] loss 5.866746
[173,   80] loss 5.558429
[173,   90] loss 5.235985
[173,  100] loss 5.196050
[173,  110] loss 4.809318
[173,  120] loss 6.297132
[173,  130] loss 5.393240
[173,  140] loss 5.846194
[173,  150] loss 5.324928
[174,   10] loss 5.440222
[174,   20] loss 6.825081
[174,   30] loss 6.165753
[174,   40] loss 6.035221
[174,   50] loss 6.012511
[174,   60] loss 5.734586
[174,   70] loss 5.399723
[174,   80] loss 5.437800
[174,   90] loss 6.383749
[174,  100] loss 5.754384
[174,  110] loss 4.943160
[174,  120] loss 6.161212
[174,  130] loss 5.738404
[174,  140] loss 5.537965
[174,  150] 

[193,   80] loss 5.474579
[193,   90] loss 6.301271
[193,  100] loss 5.589885
[193,  110] loss 5.925918
[193,  120] loss 5.218895
[193,  130] loss 5.844239
[193,  140] loss 6.663557
[193,  150] loss 5.059159
[194,   10] loss 5.244735
[194,   20] loss 6.212930
[194,   30] loss 5.865795
[194,   40] loss 6.113255
[194,   50] loss 5.722258
[194,   60] loss 5.566132
[194,   70] loss 5.081286
[194,   80] loss 5.300180
[194,   90] loss 5.415591
[194,  100] loss 5.499097
[194,  110] loss 5.576057
[194,  120] loss 5.475184
[194,  130] loss 5.240987
[194,  140] loss 6.447304
[194,  150] loss 5.332337
[195,   10] loss 6.376394
[195,   20] loss 5.485626
[195,   30] loss 5.187740
[195,   40] loss 5.184371
[195,   50] loss 5.488082
[195,   60] loss 5.466424
[195,   70] loss 6.286576
[195,   80] loss 6.071296
[195,   90] loss 6.487611
[195,  100] loss 6.160853
[195,  110] loss 6.229265
[195,  120] loss 5.783240
[195,  130] loss 6.373411
[195,  140] loss 6.019572
[195,  150] loss 4.262375
[196,   10] 

In [14]:
"""
Net=Net.double()
for epoch in np.arange(200):
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         pdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
"""
        
    

[1,   10] loss 135.697252
[1,   20] loss 133.132024
[1,   30] loss 136.135922
[1,   40] loss 129.682881
[1,   50] loss 109.772958
[1,   60] loss 72.047903
[1,   70] loss 29.887831
[1,   80] loss 29.063497
[1,   90] loss 29.995792
[1,  100] loss 20.261742
[1,  110] loss 21.661340
[1,  120] loss 24.183638
[1,  130] loss 25.412789
[1,  140] loss 20.359488
[1,  150] loss 23.177008
[1,  160] loss 24.902516
[1,  170] loss 23.919290
[2,   10] loss 24.319741
[2,   20] loss 22.901749
[2,   30] loss 23.997023
[2,   40] loss 24.646098
[2,   50] loss 22.713620
[2,   60] loss 24.063744
[2,   70] loss 23.379731
[2,   80] loss 24.956067
[2,   90] loss 21.920398
[2,  100] loss 23.380223
[2,  110] loss 22.306241
[2,  120] loss 25.500539
[2,  130] loss 21.226710
[2,  140] loss 23.228914
[2,  150] loss 21.731761
[2,  160] loss 25.283682
[2,  170] loss 22.302511
[3,   10] loss 22.448867
[3,   20] loss 22.414069
[3,   30] loss 25.253190
[3,   40] loss 22.217723
[3,   50] loss 22.760524
[3,   60] loss 21.49

[19,  160] loss 13.924522
[19,  170] loss 14.042599
[20,   10] loss 13.562330
[20,   20] loss 15.551241
[20,   30] loss 12.218430
[20,   40] loss 13.343715
[20,   50] loss 12.000882
[20,   60] loss 13.653794
[20,   70] loss 13.712260
[20,   80] loss 14.475523
[20,   90] loss 13.706664
[20,  100] loss 12.953397
[20,  110] loss 11.209604
[20,  120] loss 13.458964
[20,  130] loss 12.672550
[20,  140] loss 12.432274
[20,  150] loss 12.071071
[20,  160] loss 11.242021
[20,  170] loss 13.148363
[21,   10] loss 12.265588
[21,   20] loss 14.246901
[21,   30] loss 12.614935
[21,   40] loss 13.225317
[21,   50] loss 12.237779
[21,   60] loss 13.127100
[21,   70] loss 13.382319
[21,   80] loss 11.094586
[21,   90] loss 11.673288
[21,  100] loss 13.170361
[21,  110] loss 10.832866
[21,  120] loss 11.703973
[21,  130] loss 13.226869
[21,  140] loss 13.326879
[21,  150] loss 12.512204
[21,  160] loss 12.377491
[21,  170] loss 14.601340
[22,   10] loss 11.245708
[22,   20] loss 12.250089
[22,   30] l

[38,  110] loss 9.298384
[38,  120] loss 11.270847
[38,  130] loss 9.427728
[38,  140] loss 9.343328
[38,  150] loss 11.064446
[38,  160] loss 11.588857
[38,  170] loss 11.064279
[39,   10] loss 11.034517
[39,   20] loss 9.646738
[39,   30] loss 8.797530
[39,   40] loss 9.480678
[39,   50] loss 11.344257
[39,   60] loss 8.986026
[39,   70] loss 8.933411
[39,   80] loss 8.601417
[39,   90] loss 10.949149
[39,  100] loss 8.887711
[39,  110] loss 9.475620
[39,  120] loss 9.215676
[39,  130] loss 9.943783
[39,  140] loss 9.543800
[39,  150] loss 8.747755
[39,  160] loss 9.369966
[39,  170] loss 10.664827
[40,   10] loss 8.903946
[40,   20] loss 9.317942
[40,   30] loss 8.713625
[40,   40] loss 8.066309
[40,   50] loss 11.357366
[40,   60] loss 8.687782
[40,   70] loss 8.788661
[40,   80] loss 10.055127
[40,   90] loss 9.626108
[40,  100] loss 8.375476
[40,  110] loss 9.411304
[40,  120] loss 8.170592
[40,  130] loss 9.732172
[40,  140] loss 9.412875
[40,  150] loss 8.699495
[40,  160] loss

[57,  150] loss 8.108538
[57,  160] loss 9.242721
[57,  170] loss 9.161020
[58,   10] loss 7.516247
[58,   20] loss 8.974320
[58,   30] loss 8.663079
[58,   40] loss 7.712342
[58,   50] loss 8.125576
[58,   60] loss 9.403613
[58,   70] loss 7.974076
[58,   80] loss 8.885932
[58,   90] loss 8.038483
[58,  100] loss 7.983605
[58,  110] loss 8.662123
[58,  120] loss 8.336739
[58,  130] loss 8.139339
[58,  140] loss 8.484740
[58,  150] loss 7.363098
[58,  160] loss 8.085381
[58,  170] loss 8.411115
[59,   10] loss 7.903927
[59,   20] loss 8.191833
[59,   30] loss 7.175137
[59,   40] loss 7.613456
[59,   50] loss 8.389286
[59,   60] loss 8.418509
[59,   70] loss 7.323699
[59,   80] loss 9.322407
[59,   90] loss 7.980227
[59,  100] loss 8.273139
[59,  110] loss 8.389001
[59,  120] loss 8.095364
[59,  130] loss 7.423833
[59,  140] loss 8.493664
[59,  150] loss 8.139784
[59,  160] loss 7.106654
[59,  170] loss 6.349556
[60,   10] loss 7.031776
[60,   20] loss 7.573493
[60,   30] loss 7.702287


[77,   30] loss 7.262767
[77,   40] loss 8.921905
[77,   50] loss 6.851631
[77,   60] loss 8.246371
[77,   70] loss 7.459917
[77,   80] loss 8.017542
[77,   90] loss 7.043911
[77,  100] loss 7.925376
[77,  110] loss 7.369629
[77,  120] loss 7.394084
[77,  130] loss 7.540774
[77,  140] loss 7.571647
[77,  150] loss 6.443121
[77,  160] loss 7.181990
[77,  170] loss 6.650405
[78,   10] loss 7.533044
[78,   20] loss 6.980508
[78,   30] loss 7.050739
[78,   40] loss 7.557870
[78,   50] loss 6.161668
[78,   60] loss 6.888590
[78,   70] loss 7.257004
[78,   80] loss 6.601089
[78,   90] loss 7.045257
[78,  100] loss 6.418453
[78,  110] loss 8.009116
[78,  120] loss 8.303718
[78,  130] loss 7.359810
[78,  140] loss 7.614374
[78,  150] loss 7.061327
[78,  160] loss 7.133518
[78,  170] loss 6.795486
[79,   10] loss 7.808208
[79,   20] loss 6.549506
[79,   30] loss 7.150075
[79,   40] loss 6.353270
[79,   50] loss 8.184090
[79,   60] loss 6.534104
[79,   70] loss 6.877982
[79,   80] loss 7.682917


...

[115,   30] loss 7.137073
[115,   40] loss 6.955265
[115,   50] loss 6.128887
[115,   60] loss 6.139964
[115,   70] loss 7.075716
[115,   80] loss 6.526755
[115,   90] loss 6.704884
[115,  100] loss 5.710686
[115,  110] loss 6.070033
[115,  120] loss 5.945437
[115,  130] loss 6.155939
[115,  140] loss 7.068326
[115,  150] loss 5.769602
[115,  160] loss 6.508222
[115,  170] loss 6.391322
[116,   10] loss 7.331151
[116,   20] loss 7.401628
[116,   30] loss 8.380001
[116,   40] loss 6.139491
[116,   50] loss 6.358910
[116,   60] loss 6.947055
[116,   70] loss 7.304706
[116,   80] loss 6.941546
[116,   90] loss 8.663567
[116,  100] loss 6.777684
[116,  110] loss 7.756834
[116,  120] loss 5.584103
[116,  130] loss 6.354272
[116,  140] loss 7.103147
[116,  150] loss 8.088211
[116,  160] loss 6.494168
[116,  170] loss 6.894973
[117,   10] loss 6.412384
[117,   20] loss 6.643133
[117,   30] loss 6.399641
[117,   40] loss 6.733618
[117,   50] loss 5.912827
[117,   60] loss 7.060580
[117,   70] 

[133,  130] loss 6.434044
[133,  140] loss 6.024354
[133,  150] loss 6.707373
[133,  160] loss 7.420277
[133,  170] loss 6.826393
[134,   10] loss 5.477115
[134,   20] loss 5.724897
[134,   30] loss 6.256996
[134,   40] loss 6.877833
[134,   50] loss 5.091421
[134,   60] loss 5.491684
[134,   70] loss 6.435145
[134,   80] loss 6.150859
[134,   90] loss 7.009950
[134,  100] loss 6.664932
[134,  110] loss 7.497651
[134,  120] loss 6.385331
[134,  130] loss 5.779746
[134,  140] loss 5.842848
[134,  150] loss 6.517346
[134,  160] loss 6.391052
[134,  170] loss 5.997612
[135,   10] loss 5.335501
[135,   20] loss 6.655337
[135,   30] loss 5.224391
[135,   40] loss 5.574886
[135,   50] loss 6.059815
[135,   60] loss 6.211973
[135,   70] loss 5.490869
[135,   80] loss 8.165378
[135,   90] loss 7.381414
[135,  100] loss 6.870252
[135,  110] loss 6.581546
[135,  120] loss 6.425085
[135,  130] loss 5.865808
[135,  140] loss 5.933381
[135,  150] loss 6.184512
[135,  160] loss 6.381090
[135,  170] 

[152,   60] loss 5.848314
[152,   70] loss 5.631811
[152,   80] loss 6.097747
[152,   90] loss 5.668397
[152,  100] loss 5.364947
[152,  110] loss 4.856708
[152,  120] loss 6.031970
[152,  130] loss 6.162993
[152,  140] loss 6.695842
[152,  150] loss 5.741028
[152,  160] loss 6.546653
[152,  170] loss 6.427104
[153,   10] loss 6.613009
[153,   20] loss 5.925590
[153,   30] loss 5.396005
[153,   40] loss 5.089984
[153,   50] loss 6.264918
[153,   60] loss 6.802415
[153,   70] loss 6.016186
[153,   80] loss 6.211867
[153,   90] loss 5.509678
[153,  100] loss 6.040622
[153,  110] loss 5.893643
[153,  120] loss 6.409048
[153,  130] loss 6.316486
[153,  140] loss 6.573269
[153,  150] loss 5.540112
[153,  160] loss 6.754286
[153,  170] loss 5.680800
[154,   10] loss 5.661128
[154,   20] loss 5.573392
[154,   30] loss 6.283805
[154,   40] loss 5.510541
[154,   50] loss 6.313248
[154,   60] loss 5.982582
[154,   70] loss 6.141028
[154,   80] loss 6.768908
[154,   90] loss 5.275061
[154,  100] 

[170,  160] loss 5.841890
[170,  170] loss 6.149915
[171,   10] loss 5.449292
[171,   20] loss 6.514303
[171,   30] loss 5.624129
[171,   40] loss 5.158750
[171,   50] loss 6.471049
[171,   60] loss 5.264100
[171,   70] loss 6.633913
[171,   80] loss 5.428715
[171,   90] loss 6.122781
[171,  100] loss 6.021783
[171,  110] loss 6.335677
[171,  120] loss 5.127951
[171,  130] loss 6.054306
[171,  140] loss 6.861214
[171,  150] loss 5.712167
[171,  160] loss 5.518883
[171,  170] loss 5.466637
[172,   10] loss 6.419921
[172,   20] loss 5.395911
[172,   30] loss 5.360647
[172,   40] loss 5.195174
[172,   50] loss 5.198094
[172,   60] loss 5.872464
[172,   70] loss 5.698540
[172,   80] loss 5.972355
[172,   90] loss 5.701766
[172,  100] loss 5.515316
[172,  110] loss 6.433910
[172,  120] loss 5.423423
[172,  130] loss 4.583866
[172,  140] loss 6.533761
[172,  150] loss 6.168923
[172,  160] loss 5.717466
[172,  170] loss 6.295311
[173,   10] loss 5.561753
[173,   20] loss 5.868243
[173,   30] 

[189,   90] loss 5.965511
[189,  100] loss 5.648822
[189,  110] loss 5.928625
[189,  120] loss 6.260511
[189,  130] loss 5.295016
[189,  140] loss 5.573620
[189,  150] loss 5.550615
[189,  160] loss 5.453101
[189,  170] loss 5.802822
[190,   10] loss 5.424704
[190,   20] loss 6.064295
[190,   30] loss 6.673784
[190,   40] loss 6.554471
[190,   50] loss 6.219193
[190,   60] loss 5.802653
[190,   70] loss 5.160568
[190,   80] loss 6.200963
[190,   90] loss 6.041080
[190,  100] loss 5.828318
[190,  110] loss 6.043290
[190,  120] loss 5.887069
[190,  130] loss 5.974542
[190,  140] loss 6.323385
[190,  150] loss 5.244750
[190,  160] loss 5.875966
[190,  170] loss 6.290674
[191,   10] loss 5.574545
[191,   20] loss 5.038197
[191,   30] loss 4.771653
[191,   40] loss 5.576842
[191,   50] loss 6.187910
[191,   60] loss 6.340060
[191,   70] loss 5.169199
[191,   80] loss 5.606100
[191,   90] loss 6.136285
[191,  100] loss 5.526590
[191,  110] loss 5.794770
[191,  120] loss 4.548505
[191,  130] 

In [ ]:
"""
Net=Net.double()
for epoch in np.arange(200):
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         pdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
"""

[1,   10] loss 136.703597
[1,   20] loss 138.798358
[1,   30] loss 134.008359
[1,   40] loss 133.126934
[1,   50] loss 134.270320
[1,   60] loss 134.632149
[1,   70] loss 136.579115
[1,   80] loss 131.926537
[1,   90] loss 136.188359
[1,  100] loss 133.792999
[1,  110] loss 135.169166
[1,  120] loss 137.343139
[1,  130] loss 135.310419
[1,  140] loss 135.593038
[1,  150] loss 132.522017
[1,  160] loss 129.073107
[1,  170] loss 132.600764
[2,   10] loss 127.288244
[2,   20] loss 127.277590
[2,   30] loss 130.485830
[2,   40] loss 126.587347
[2,   50] loss 124.986941
[2,   60] loss 122.629768
[2,   70] loss 118.807804
[2,   80] loss 118.591237
[2,   90] loss 114.225952
[2,  100] loss 111.202298
[2,  110] loss 103.559523
[2,  120] loss 100.465943
[2,  130] loss 88.555997
[2,  140] loss 77.186387
[2,  150] loss 67.908691
[2,  160] loss 61.178656
[2,  170] loss 48.983955
[3,   10] loss 34.545110
[3,   20] loss 25.086295
[3,   30] loss 23.647689
[3,   40] loss 22.993890
[3,   50] loss 23.840

[19,  150] loss 22.158142
[19,  160] loss 21.742194
[19,  170] loss 21.026423
[20,   10] loss 21.711406
[20,   20] loss 20.858520
[20,   30] loss 23.239516
[20,   40] loss 24.091834
[20,   50] loss 20.484090
[20,   60] loss 23.778143
[20,   70] loss 20.266074
[20,   80] loss 22.813613
[20,   90] loss 22.190020
[20,  100] loss 20.583548
[20,  110] loss 25.000506
[20,  120] loss 22.701241
[20,  130] loss 21.565656
[20,  140] loss 23.338614
[20,  150] loss 18.598208
[20,  160] loss 22.733531
[20,  170] loss 21.846018
[21,   10] loss 21.150590
[21,   20] loss 22.311897
[21,   30] loss 21.728086
[21,   40] loss 25.154737
[21,   50] loss 21.852775
[21,   60] loss 22.690744
[21,   70] loss 20.490834
[21,   80] loss 22.538394
[21,   90] loss 25.012417
[21,  100] loss 20.654424
[21,  110] loss 25.422486
[21,  120] loss 22.617888
[21,  130] loss 22.940664
[21,  140] loss 21.983016
[21,  150] loss 18.692820
[21,  160] loss 19.178865
[21,  170] loss 22.494150
[22,   10] loss 20.328596
[22,   20] l

[38,   80] loss 21.911967
[38,   90] loss 20.969566
[38,  100] loss 23.621026
[38,  110] loss 22.839186
[38,  120] loss 21.359365
[38,  130] loss 20.356863
[38,  140] loss 19.717113
[38,  150] loss 20.182249
[38,  160] loss 21.717815
[38,  170] loss 23.206685


In [42]:
"""
##################
# Add Batchnorm
#################
Net=Net.double()
for epoch in np.arange(200):
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         pdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
"""   

[1,   10] loss 133.510298
[1,   20] loss 129.210721
[1,   30] loss 128.971799
[1,   40] loss 129.533647
[1,   50] loss 128.910575
[1,   60] loss 119.861357
[1,   70] loss 120.486735
[1,   80] loss 121.786363
[1,   90] loss 123.863312
[1,  100] loss 118.306047
[1,  110] loss 119.703260
[1,  120] loss 118.047237
[1,  130] loss 112.544813
[1,  140] loss 112.393141
[1,  150] loss 111.408859
[1,  160] loss 110.383351
[1,  170] loss 112.076205
[2,   10] loss 110.336740
[2,   20] loss 115.174991
[2,   30] loss 108.936065
[2,   40] loss 111.875760
[2,   50] loss 109.404557
[2,   60] loss 104.851883
[2,   70] loss 108.941197
[2,   80] loss 110.627535
[2,   90] loss 103.881279
[2,  100] loss 110.637222
[2,  110] loss 105.971035
[2,  120] loss 106.214590
[2,  130] loss 106.739042
[2,  140] loss 101.028365
[2,  150] loss 103.038653
[2,  160] loss 101.415159
[2,  170] loss 103.089151
[3,   10] loss 103.558778
[3,   20] loss 105.532187
[3,   30] loss 101.258865
[3,   40] loss 99.649738
[3,   50] los

[19,  150] loss 34.156368
[19,  160] loss 29.198016
[19,  170] loss 22.629171
[20,   10] loss 24.296333
[20,   20] loss 26.363209
[20,   30] loss 21.619981
[20,   40] loss 25.070770
[20,   50] loss 23.200289
[20,   60] loss 22.200354
[20,   70] loss 24.907046
[20,   80] loss 21.240783
[20,   90] loss 24.810225
[20,  100] loss 26.976460
[20,  110] loss 20.933960
[20,  120] loss 23.674601
[20,  130] loss 22.046106
[20,  140] loss 25.296936
[20,  150] loss 20.600200
[20,  160] loss 20.108168
[20,  170] loss 20.355912
[21,   10] loss 20.819691
[21,   20] loss 20.237694
[21,   30] loss 23.057924
[21,   40] loss 21.792635
[21,   50] loss 18.685899
[21,   60] loss 16.875513
[21,   70] loss 17.426322
[21,   80] loss 21.659650
[21,   90] loss 16.467097
[21,  100] loss 19.748865
[21,  110] loss 18.289237
[21,  120] loss 19.537880
[21,  130] loss 18.984454
[21,  140] loss 19.014535
[21,  150] loss 18.497637
[21,  160] loss 19.766049
[21,  170] loss 15.095043
[22,   10] loss 16.770275
[22,   20] l

[38,  160] loss 7.291692
[38,  170] loss 8.183264
[39,   10] loss 6.451403
[39,   20] loss 7.588937
[39,   30] loss 8.072056
[39,   40] loss 8.074267
[39,   50] loss 6.597406
[39,   60] loss 7.607762
[39,   70] loss 7.607765
[39,   80] loss 6.796007
[39,   90] loss 8.057531
[39,  100] loss 8.423830
[39,  110] loss 8.497833
[39,  120] loss 7.612487
[39,  130] loss 7.592567
[39,  140] loss 8.319802
[39,  150] loss 8.690277
[39,  160] loss 7.535785
[39,  170] loss 8.473172
[40,   10] loss 7.409886
[40,   20] loss 7.412946
[40,   30] loss 6.937875
[40,   40] loss 7.046149
[40,   50] loss 7.369888
[40,   60] loss 7.274832
[40,   70] loss 8.103117
[40,   80] loss 6.964518
[40,   90] loss 7.268287
[40,  100] loss 7.307193
[40,  110] loss 6.300410
[40,  120] loss 7.921124
[40,  130] loss 8.098767
[40,  140] loss 6.809715
[40,  150] loss 7.999885
[40,  160] loss 7.408342
[40,  170] loss 7.318217
[41,   10] loss 8.017296
[41,   20] loss 7.007363
[41,   30] loss 6.927276
[41,   40] loss 6.366526


[58,   40] loss 6.212299
[58,   50] loss 6.789189
[58,   60] loss 6.984483
[58,   70] loss 7.705182
[58,   80] loss 6.451768
[58,   90] loss 6.165674
[58,  100] loss 6.612004
[58,  110] loss 5.875752
[58,  120] loss 6.545681
[58,  130] loss 7.532174
[58,  140] loss 8.110649
[58,  150] loss 6.519472
[58,  160] loss 7.278420
[58,  170] loss 7.256977
[59,   10] loss 5.596201
[59,   20] loss 6.067399
[59,   30] loss 6.526081
[59,   40] loss 6.814282
[59,   50] loss 6.173669
[59,   60] loss 6.606664
[59,   70] loss 6.313829
[59,   80] loss 6.733683
[59,   90] loss 6.795262
[59,  100] loss 7.275730
[59,  110] loss 7.115267
[59,  120] loss 6.703414
[59,  130] loss 5.871368
[59,  140] loss 6.558937
[59,  150] loss 6.599273
[59,  160] loss 6.786108
[59,  170] loss 7.472330
[60,   10] loss 6.663838
[60,   20] loss 7.083605
[60,   30] loss 6.117562
[60,   40] loss 6.848421
[60,   50] loss 5.835079
[60,   60] loss 5.419387
[60,   70] loss 6.519651
[60,   80] loss 6.181215
[60,   90] loss 6.532900


[77,   90] loss 6.397453
[77,  100] loss 5.404701
[77,  110] loss 6.100868
[77,  120] loss 6.599023
[77,  130] loss 5.636350
[77,  140] loss 7.506853
[77,  150] loss 6.524572
[77,  160] loss 6.223919
[77,  170] loss 7.314441
[78,   10] loss 6.942189
[78,   20] loss 7.149218
[78,   30] loss 5.894286
[78,   40] loss 7.202015
[78,   50] loss 6.972951
[78,   60] loss 6.497607
[78,   70] loss 6.405176
[78,   80] loss 5.433015
[78,   90] loss 6.877333
[78,  100] loss 6.272793
[78,  110] loss 6.725142
[78,  120] loss 6.853329
[78,  130] loss 6.569710
[78,  140] loss 6.624628
[78,  150] loss 6.514931
[78,  160] loss 6.066927
[78,  170] loss 6.465120
[79,   10] loss 6.450638
[79,   20] loss 6.413636
[79,   30] loss 6.269304
[79,   40] loss 6.612245
[79,   50] loss 6.744072
[79,   60] loss 6.367511
[79,   70] loss 5.979520
[79,   80] loss 6.758767
[79,   90] loss 6.545367
[79,  100] loss 6.724061
[79,  110] loss 6.481737
[79,  120] loss 6.151715
[79,  130] loss 6.803194
[79,  140] loss 6.915249


...

[115,   90] loss 6.356405
[115,  100] loss 6.199996
[115,  110] loss 6.118873
[115,  120] loss 6.128287
[115,  130] loss 6.454191
[115,  140] loss 6.388371
[115,  150] loss 5.796865
[115,  160] loss 6.224969
[115,  170] loss 6.401345
[116,   10] loss 5.628495
[116,   20] loss 6.768804
[116,   30] loss 5.613882
[116,   40] loss 7.580664
[116,   50] loss 5.707118
[116,   60] loss 5.423428
[116,   70] loss 6.350463
[116,   80] loss 6.353670
[116,   90] loss 6.038321
[116,  100] loss 6.125195
[116,  110] loss 6.443317
[116,  120] loss 5.875223
[116,  130] loss 7.237416
[116,  140] loss 6.453495
[116,  150] loss 6.694553
[116,  160] loss 6.337698
[116,  170] loss 6.395854
[117,   10] loss 5.620164
[117,   20] loss 5.995029
[117,   30] loss 6.376668
[117,   40] loss 5.753388
[117,   50] loss 5.611939
[117,   60] loss 5.770703
[117,   70] loss 6.241664
[117,   80] loss 6.927908
[117,   90] loss 6.078253
[117,  100] loss 6.043588
[117,  110] loss 6.132601
[117,  120] loss 5.315321
[117,  130] 

[134,   20] loss 6.600333
[134,   30] loss 6.177122
[134,   40] loss 5.462709
[134,   50] loss 5.482911
[134,   60] loss 6.180531
[134,   70] loss 6.418016
[134,   80] loss 6.405883
[134,   90] loss 5.305753
[134,  100] loss 5.584687
[134,  110] loss 4.845298
[134,  120] loss 5.617019
[134,  130] loss 7.097704
[134,  140] loss 5.609762
[134,  150] loss 6.582255
[134,  160] loss 6.165132
[134,  170] loss 6.207636
[135,   10] loss 6.068195
[135,   20] loss 6.826897
[135,   30] loss 5.550401
[135,   40] loss 5.795979
[135,   50] loss 6.414813
[135,   60] loss 5.603490
[135,   70] loss 6.029291
[135,   80] loss 5.766794
[135,   90] loss 5.793034
[135,  100] loss 5.480529
[135,  110] loss 6.499504
[135,  120] loss 6.497374
[135,  130] loss 6.324396
[135,  140] loss 5.083553
[135,  150] loss 7.036065
[135,  160] loss 6.164377
[135,  170] loss 5.661274
[136,   10] loss 6.149521
[136,   20] loss 6.036883
[136,   30] loss 5.557001
[136,   40] loss 5.617517
[136,   50] loss 6.758199
[136,   60] 

[152,  120] loss 5.125784
[152,  130] loss 5.780512
[152,  140] loss 5.829846
[152,  150] loss 5.792381
[152,  160] loss 5.894352
[152,  170] loss 6.903598
[153,   10] loss 5.788678
[153,   20] loss 5.669022
[153,   30] loss 6.324288
[153,   40] loss 4.731528
[153,   50] loss 5.269047
[153,   60] loss 6.103747
[153,   70] loss 5.685924
[153,   80] loss 6.314313
[153,   90] loss 5.745398
[153,  100] loss 5.967002
[153,  110] loss 5.888975
[153,  120] loss 5.609375
[153,  130] loss 6.439041
[153,  140] loss 6.026294
[153,  150] loss 6.316552
[153,  160] loss 6.013807
[153,  170] loss 5.531614
[154,   10] loss 6.574974
[154,   20] loss 5.779915
[154,   30] loss 6.639457
[154,   40] loss 5.343589
[154,   50] loss 5.823604
[154,   60] loss 6.841971
[154,   70] loss 5.677111
[154,   80] loss 6.258836
[154,   90] loss 5.900847
[154,  100] loss 5.279575
[154,  110] loss 6.233979
[154,  120] loss 5.816915
[154,  130] loss 5.490392
[154,  140] loss 6.404129
[154,  150] loss 5.476920
[154,  160] 

[171,   50] loss 6.875759
[171,   60] loss 5.618118
[171,   70] loss 5.687065
[171,   80] loss 6.116016
[171,   90] loss 6.125050
[171,  100] loss 6.267258
[171,  110] loss 5.414997
[171,  120] loss 6.307967
[171,  130] loss 6.161042
[171,  140] loss 5.480531
[171,  150] loss 5.970293
[171,  160] loss 5.606150
[171,  170] loss 5.735455
[172,   10] loss 5.432514
[172,   20] loss 6.095217
[172,   30] loss 5.602240
[172,   40] loss 5.919826
[172,   50] loss 5.460383
[172,   60] loss 4.946956
[172,   70] loss 5.499675
[172,   80] loss 6.528461
[172,   90] loss 5.837299
[172,  100] loss 5.758819
[172,  110] loss 5.145051
[172,  120] loss 5.038379
[172,  130] loss 6.325766
[172,  140] loss 5.195826
[172,  150] loss 5.668924
[172,  160] loss 5.685100
[172,  170] loss 5.422951
[173,   10] loss 5.672583
[173,   20] loss 5.897964
[173,   30] loss 6.284618
[173,   40] loss 5.662013
[173,   50] loss 5.300467
[173,   60] loss 6.166838
[173,   70] loss 5.781734
[173,   80] loss 5.739703
[173,   90] 

[189,  150] loss 5.411120
[189,  160] loss 5.189910
[189,  170] loss 5.525759
[190,   10] loss 6.786844
[190,   20] loss 5.948250
[190,   30] loss 4.996490
[190,   40] loss 5.997247
[190,   50] loss 5.959597
[190,   60] loss 5.008039
[190,   70] loss 5.481594
[190,   80] loss 6.363056
[190,   90] loss 5.469138
[190,  100] loss 5.160340
[190,  110] loss 5.440045
[190,  120] loss 5.895744
[190,  130] loss 5.151319
[190,  140] loss 6.018259
[190,  150] loss 6.351593
[190,  160] loss 5.159906
[190,  170] loss 5.698419
[191,   10] loss 5.801672
[191,   20] loss 6.409306
[191,   30] loss 5.247163
[191,   40] loss 6.436349
[191,   50] loss 5.729051
[191,   60] loss 6.214496
[191,   70] loss 5.849782
[191,   80] loss 5.286527
[191,   90] loss 6.798614
[191,  100] loss 5.715291
[191,  110] loss 6.380467
[191,  120] loss 5.458384
[191,  130] loss 5.665997
[191,  140] loss 5.234845
[191,  150] loss 4.777835
[191,  160] loss 5.366889
[191,  170] loss 5.834747
[192,   10] loss 5.489649
[192,   20] 

In [48]:
# t.save(Net.state_dict(),'/home/wcj/PytorchDemo/checkpoints/Net_epoch%s_batchsize20.pth'%(epoch+1))

In [34]:
'''
Net=PPGNet(7,7).eval()
Net.double()
Net.load_state_dict(t.load('/home/wcj/PytorchDemo/checkpoints/Net_epoch200_batchsize20.pth'))

testfilepath='/home/wcj/PytorchDemo/TestData'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
    #         pdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))
'''

6: 1268个样本计算MAE: 18.912492


In [36]:
#############################
# test on original dataset
#############################
"""
sub=0
total=0
with t.no_grad():
    for data in dloador:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
    #         pdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print('%d个样本计算MAE: %.6f'%(total,sub/total))
"""

1696个样本计算MAE: 11.826419


In [15]:
#######################
# test on leave one
#######################
Net=PPGNet(7,7).eval()
Net.double()
Net.load_state_dict(t.load('/home/wcj/PytorchDemo/checkpoints/Net_epoch200_LeaveOne.pth'))

testfilepath='/home/wcj/PytorchDemo/TestDataLeaveOne'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
    #         pdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print('Test on LeaveOne %d个样本计算MAE: %.6f'%(total,sub/total))




Test on LeaveOne 140个样本计算MAE: 8.425315
